## Decision Tree for Spam 
### (make sure to have run featurize_spam.py)

In [1]:
import numpy as np
np.random.seed(1)

import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from decisiontree import *

In [2]:
spam_path = "datasets/spam-dataset/spam_data.mat"
data = scipy.io.loadmat(spam_path)
x = data["training_data"]
x = StandardScaler().fit_transform(x)
pca = PCA(n_components=0.1)
pca_fit = pca.fit(x)
x = pca_fit.transform(x)
y = np.squeeze(data["training_labels"])
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
# x_test = data["test_data"]
x_test = pca_fit.transform(data["test_data"])
x.shape,y.shape,x_test.shape

((3336, 4), (3336,), (5857, 4))

In [3]:
%%time
dt = DecisionTree(max_depth=5,header="spam_decisiontree")
dt.fit(x_train, y_train)

CPU times: user 27.5 s, sys: 145 ms, total: 27.6 s
Wall time: 27.1 s


In [4]:
%%time
train_preds = dt.predict(x_train,"train")
print("training accuracy:",np.mean(train_preds==y_train))
val_preds = dt.predict(x_val,"val")
print("validation accuracy:",np.mean(val_preds==y_val))
dt.predict(x_test,"test")

training accuracy: 0.8024737631184408
validation accuracy: 0.7679640718562875
saved predictions
CPU times: user 87.8 ms, sys: 3.14 ms, total: 91 ms
Wall time: 90.4 ms
